# Title: msticpy - nbwidgets
## Description:
This contains a few aggregated widgets using IPyWidgets that help speed things up during an investigation.



<a id='contents'></a>
## Table of Contents
- [Setting query start/end times](#QueryTime)
- [Simple time range](#Lookback)
- [Selecting and Displaying Alerts](#AlertSelector)
- [Selecting from list or dict](#SelectString)
- [Getting a value from environment](#GetEnvironmentKey)


In [24]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

from IPython.display import display, Markdown
import pandas as pd
# Import nbtools package
import msticpy.nbtools as nbtools


<a id='QueryTime'></a>[Contents](#contents)
## QueryTime

This widget is used to specify time boundaries - designed to be used with the built-in msticpy queries and custom queries.
The `start` and `end` times are exposed as datetime properties.

```
QueryTime.

Composite widget to capture date and time origin
and set start and end times for queries.

Parameters
----------
QueryParamProvider : QueryParamProvider
    Abstract base class

Parameters
----------
origin_time : datetime, optional
    The origin time (the default is `datetime.utcnow()`)
label : str, optional
    The description to display
    (the default is 'Select time ({units}) to look back')
before : int, optional
    The default number of `units` before the `origin_time`
    (the default is 60)
after : int, optional
    The default number of `units` after the `origin_time`
    (the default is 10)
max_before : int, optional
    The largest value for `before` (the default is 600)
max_after : int, optional
    The largest value for `after` (the default is 100)
units : str, optional
    Time unit (the default is 'min')
    Permissable values are 'day', 'hour', 'minute', 'second'
    These can all be abbreviated down to initial characters
    ('d', 'm', etc.)
auto_display : bool, optional
    Whether to display on instantiation (the default is False)
```

In [2]:
q_times = nbtools.QueryTime(units='day', max_before=20, before=5, max_after=1)
q_times.display()

HTML(value='<h4>Set query time boundaries</h4>')

In [3]:
print(q_times.start, '....', q_times.end)

2019-03-10 19:57:53.694691 .... 2019-03-16 19:57:53.694691


Keep multiple query boundaries aligged by having QueryTime instances reference the time of the same alert or event.

In [4]:
from datetime import datetime, timedelta
class MyAlert:
    pass
alert = MyAlert()
alert.TimeGenerated = datetime.utcnow() - timedelta(15)
alert.TimeGenerated

q_times1 = nbtools.QueryTime(units='hour', max_before=20, before=1, max_after=1, 
                             origin_time=alert.TimeGenerated, auto_display=True)

q_times2 = nbtools.QueryTime(units='hour', max_before=20, before=4, max_after=2, 
                             origin_time=alert.TimeGenerated, auto_display=True)

HTML(value='<h4>Set query time boundaries</h4>')

HTML(value='<h4>Set query time boundaries</h4>')

In [5]:
alert.TimeGenerated = datetime.utcnow()
q_times1 = nbtools.QueryTime(units='hour', max_before=20, before=1, max_after=1, 
                             origin_time=alert.TimeGenerated, auto_display=True)

q_times2 = nbtools.QueryTime(units='hour', max_before=20, before=4, max_after=2, 
                             origin_time=alert.TimeGenerated, auto_display=True)

HTML(value='<h4>Set query time boundaries</h4>')

HTML(value='<h4>Set query time boundaries</h4>')

In [6]:
# Use in a query
my_kql = f'''
SecurityAlert 
| where TimeGenerated >= {q_times1.start} 
| where TimeGenerated <= {q_times1.end}'''
print(my_kql)


SecurityAlert 
| where TimeGenerated >= 2019-03-15 18:57:55.701709 
| where TimeGenerated <= 2019-03-15 20:57:55.701709


In [7]:
nbtools.qry.show_filled_query('list_alerts', provs=[q_times2]);

SecurityAlert
| project
TenantId,
StartTimeUtc = StartTime,
EndTimeUtc = EndTime,
ProviderAlertId = VendorOriginalId,
SystemAlertId,
ProviderName,
VendorName,
AlertType = AlertName,
AlertName,
AlertDisplayName = DisplayName,
Description,
Severity = AlertSeverity,
IsIncident,
ExtendedProperties,
Entities,
ConfidenceLevel,
ConfidenceScore,
ExtendedLinks,
WorkspaceSubscriptionId,
WorkspaceResourceGroup,
TimeGenerated,
ResourceId,
SourceComputerId
| where TimeGenerated >= datetime(2019-03-15 15:57:55.701709)
| where TimeGenerated <= datetime(2019-03-15 21:57:55.701709)
| extend extendedProps = parse_json(ExtendedProperties)
| extend CompromisedEntity = tostring(extendedProps['Compromised Host'])
| project-away extendedProps


<a id='Lookback'></a>[Contents](#contents)
## Lookback
Simpler version with single slider value

Docstring:
`nbtools.Lookback?`

In [8]:
alert.TimeGenerated = datetime.utcnow() - timedelta(5)
lb = nbtools.Lookback(origin_time=alert.TimeGenerated, auto_display=True, max_value=48)

IntSlider(value=4, description='Select time (hour) to look back', layout=Layout(height='50px', width='60%'), m…

In [10]:
print(lb.start, '....', lb.end)

2019-03-10 15:57:57.827783 .... 2019-03-10 19:57:57.827783


<a id='AlertSelector'></a>[Contents](#contents)
## AlertSelector

```
AlertSelector.

View list of alerts and select one for investigation.
Optionally provide and action to call with the selected alert as a parameter
(typically used to display the alert.)

Attributes:
    selected_alert: the selected alert
    alert_id: the ID of the selected alert
    alerts: the current alert list (DataFrame)
Init docstring:
Create a new instance of AlertSelector.

Parameters
----------
alerts : pd.DataFrame
    DataFrame of alerts.
action : Callable[..., None], optional
    Optional function to execute for each selected alert.
    (the default is None)
columns : list, optional
    Override the default column names to use from `alerts`
    (the default is ['StartTimeUtc', 'AlertName',
    'CompromisedEntity', 'SystemAlertId'])
auto_display : bool, optional
    Whether to display on instantiation (the default is False)
```

In [26]:
# Load test data
alerts = pd.read_csv('data/alertlist.csv')

display(Markdown('### Simple alert selector'))
alert_select = nbtools.AlertSelector(alerts=alerts)
alert_select.display()

### Simple alert selector

In [27]:
import msticpy.nbtools.nbdisplay as nbdisp
alert_select = nbtools.AlertSelector(alerts=alerts, action=nbdisp.display_alert)
display(Markdown('### Alert selector with action=DisplayAlert'))
alert_select.display()

### Alert selector with action=DisplayAlert

In [31]:
def show_full_alert(selected_alert):
    global security_alert, alert_ip_entities
    security_alert = nbtools.SecurityAlert(alert_select.selected_alert)
    nbdisp.display_alert(security_alert, show_entities=True)
        
alert_select = nbtools.AlertSelector(alerts=alerts.query('CompromisedEntity == "MSTICALERTSWIN1"'), 
                                     action=show_full_alert)
display(Markdown('### Or a more detailed display with extracted entities'))
alert_select.display()

### Or a more detailed display with extracted entities

<a id='SelectString'></a>[Contents](#contents)
## SelectString

Similar to AlertSelector but simpler and allows you to use any list or dictionary of items.

```
Selection list from list or dict.

Attributes:
    value : The selected value.
Init docstring:
Select an item from a list or dict.

Parameters
----------
description : str, optional
    The widget label to display (the default is None)
item_list : List[str], optional
    A `list` of items to select from (the default is None)
item_dict : Mapping[str, str], optional
    A `dict` of items to select from. When using `item_dict`
    the keys are displayed as the selectable items and value
    corresponding to the selected key is set as the `value`
    property.
    (the default is None)
action : Callable[..., None], optional
    function to call when item selected (passed a single
    parameter - the value of the currently selected item)
    (the default is None)
auto_display : bool, optional
    Whether to display on instantiation (the default is False)
height : str, optional
    Selection list height (the default is '100px')
width : str, optional
    Selection list width (the default is '50%')
```

In [58]:
ent_dict = {ent['Type']:ent for ent in security_alert.entities}
import pprint
nbtools.SelectString(item_dict=ent_dict, description='Select an item', action=print, auto_display=True);


Select(description='Select an item', layout=Layout(height='100px', width='50%'), options=('host', 'account', '…

{ 'Host': { 'AzureID': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/providers/Microsoft.Compute/virtualMachines/MSTICAlertsWin1',
            'HostName': 'msticalertswin1',
            'OMSAgentID': '46fe7078-61bb-4bed-9430-7ac01d91c273',
            'Type': 'host'},
  'NTDomain': 'msticalertswin1',
  'Name': 'msticadmin',
  'Type': 'account'}
{ 'AzureID': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHuntOMSWorkspaceRG/providers/Microsoft.Compute/virtualMachines/MSTICAlertsWin1',
  'HostName': 'msticalertswin1',
  'OMSAgentID': '46fe7078-61bb-4bed-9430-7ac01d91c273',
  'Type': 'host'}


<a id='GetEnvironmentKey'></a>[Contents](#contents)
## GetEnvironmentKey
Get editable value of environment variable. Common use would be retrieving an API key from your environment or allowing you to paste in a value if the environment key isn't set.

Note setting the variable only persists in the python kernel process running at the time. So you can retrieve it later in the notebook but not in other processes.

In [43]:
nbtools.GetEnvironmentKey(env_var='userprofile', auto_display=True);